In [ ]:
import getpass

api_key = getpass.getpass(prompt="OpenAI API Key: ")

In [ ]:
!pip install openai

In [ ]:
import datetime
import getpass
import tempfile

import openai

CHUNK_SIZE = 50000

if "api_key" not in locals():
    api_key = getpass.getpass(prompt="OpenAI API Key: ")

client = openai.OpenAI(
    api_key=api_key,
)

batch_ids = []

f = open("input.jsonl")

lines = f.readlines()
for i in range(0, len(lines), CHUNK_SIZE):
    chunk = lines[i:i + CHUNK_SIZE]

    with tempfile.NamedTemporaryFile() as tempf:
        tempf.write("\n".join(chunk).encode("utf-8"))
        tempf.seek(0)

        input_file = client.files.create(
            file=tempf.file,
            purpose="batch",
        )

        response = client.batches.create(
            input_file_id=input_file.id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
                "chunk": f"{i // CHUNK_SIZE}",
            }
        )

        batch_ids.append(response.id)

f.close()

batch_ids_file = datetime.datetime.now().isoformat()

with open(batch_ids_file, "w") as f:
    f.write("\n".join(batch_ids))

In [ ]:
import getpass

import openai

if "api_key" not in locals():
    api_key = getpass.getpass(prompt="OpenAI API Key: ")

client = openai.OpenAI(
    api_key=api_key,
)

if "batch_ids_file" not in locals():
    batch_ids_file = input("Enter execution file: ")

f = open(batch_ids_file)

batch_ids = f.readlines()

with open(f"{batch_ids_file}.done", "wb") as w:
    for batch_id in batch_ids:
        batch = client.batches.retrieve(batch_id.strip())
        match batch.status:
            case "completed":
                w.write(client.files.content(batch.output_file_id).read())
            case "failed":
                w.write(client.files.content(batch.error_file_id).read())
            case _:
                print(batch.status)

f.close()